<a href="https://colab.research.google.com/github/terkoizmy/Titanic/blob/main/Titanic_Train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# import Library

In [148]:
pip install catboost

In [149]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from catboost import CatBoostClassifier
import joblib

# Data processing

In [150]:
dataset = pd.read_csv('train.csv')
dataset

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [151]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [152]:
dataset.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

## Data Cleaning

In [153]:
# fill in the missing value in Age column with mean
dataset['Age'] = dataset['Age'].fillna(dataset['Age'].mean())
# fill in the missing value in Embarked column with modus
dataset['Embarked'] = dataset['Embarked'].fillna(dataset['Embarked'].mode()[0])
# delete column which we don't use
dataset.drop(columns = ["Cabin", "Ticket", "PassengerId", "Name", "Fare"], inplace = True)

In [154]:
dataset.isnull().sum()

Survived    0
Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Embarked    0
dtype: int64

## Label Encoder

In [155]:
X = dataset.iloc[:,1:7]
y = dataset.iloc[:,0]

Sex_encoder = LabelEncoder().fit(X['Sex'])
X['Sex'] = Sex_encoder.transform(X['Sex'])

Sex_encoder = LabelEncoder().fit(X['Embarked'])
X['Embarked'] = Sex_encoder.transform(X['Embarked'])

## Scaling Data

In [156]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X = sc.fit_transform(X)

In [157]:
X

array([[ 0.82737724,  0.73769513, -0.5924806 ,  0.43279337, -0.47367361,
         0.58595414],
       [-1.56610693, -1.35557354,  0.63878901,  0.43279337, -0.47367361,
        -1.9423032 ],
       [ 0.82737724, -1.35557354, -0.2846632 , -0.4745452 , -0.47367361,
         0.58595414],
       ...,
       [ 0.82737724, -1.35557354,  0.        ,  0.43279337,  2.00893337,
         0.58595414],
       [-1.56610693,  0.73769513, -0.2846632 , -0.4745452 , -0.47367361,
        -1.9423032 ],
       [ 0.82737724,  0.73769513,  0.17706291, -0.4745452 , -0.47367361,
        -0.67817453]])

## Spliting data

In [159]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 123)

# Modeling Machine Learning

## Model With XGBClassifier

In [160]:
XGBmodel = XGBClassifier(n_estimators=100, max_depth=3, learning_rate=0.01)
XGBmodel.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.01, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [161]:
# make predictions for test data
XGBpredict = XGBmodel.predict(X_test)

In [162]:
# evaluate predictions
print("=== Confusion Matrix ===")
print(confusion_matrix(y_test, XGBpredict))
print('\n')
print("=== Classification Report ===")
# evaluate predictions
accuracy = accuracy_score(y_test, XGBpredict)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

=== Confusion Matrix ===
[[120  19]
 [ 17  67]]


=== Classification Report ===
Accuracy: 83.86%


In [163]:
XGBmodel

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.01, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

## Model With Random Forest

In [164]:
from sklearn import model_selection
# random forest model creation
rfc = RandomForestClassifier()
rfc.fit(X_train,y_train)
# predictions
rfc_predict = rfc.predict(X_test)

In [165]:
print("=== Confusion Matrix ===")
print(confusion_matrix(y_test, rfc_predict))
print('\n')
print("=== Classification Report ===")
# evaluate predictions
accuracy = accuracy_score(y_test, rfc_predict)
print("Accuracy: %.2f%%" % (accuracy * 100.0))


=== Confusion Matrix ===
[[121  18]
 [ 28  56]]


=== Classification Report ===
Accuracy: 79.37%


In [166]:
rfc

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

## Model With CatBoost

In [167]:
CBCmodel = CatBoostClassifier(iterations=10,
                           learning_rate=5,
                           depth=5)

In [168]:
CBCmodel.fit(X_train, y_train)

0:	learn: 0.8777079	total: 763us	remaining: 6.87ms
1:	learn: 10.1645479	total: 1.35ms	remaining: 5.41ms
2:	learn: 18.7255617	total: 2.17ms	remaining: 5.05ms
3:	learn: 13.1175414	total: 3.01ms	remaining: 4.51ms
4:	learn: 12.2357979	total: 3.77ms	remaining: 3.77ms
5:	learn: 11.3707882	total: 4.56ms	remaining: 3.04ms
6:	learn: 6.5238732	total: 5.36ms	remaining: 2.29ms
7:	learn: 4.8542079	total: 5.89ms	remaining: 1.47ms
8:	learn: 4.3347635	total: 7.27ms	remaining: 807us
9:	learn: 21.2496081	total: 8ms	remaining: 0us


learning rate is greater than 1. You probably need to decrease learning rate.
learning rate is greater than 1. You probably need to decrease learning rate.


In [171]:
X_train

array([[-0.82212766,  0.74936248],
       [ 1.76819041,  0.86517753],
       [-0.39346309, -0.89918468],
       ...,
       [-0.49455932, -1.28354791],
       [-0.42055595, -1.32494954],
       [-0.54146984, -0.81638143]])

In [169]:
CBCpredict = model.predict(X_test)

ValueError: ignored

In [ ]:
print("=== Confusion Matrix ===")
print(confusion_matrix(y_test, CBCpredict))
print('\n')
print("=== Classification Report ===")
# evaluate predictions
accuracy = accuracy_score(y_test, CBCpredict)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

# Train full data

In [170]:
# XGBoost
XGB_model = XGBClassifier()
XGB_model.fit(X, y)

# CatBoost
CBC_model = CatBoostClassifier(iterations=10,
                           learning_rate=5,
                           depth=5)
CBC_model.fit(X_train, y_train)

# RandomForest
rfc_model = RandomForestClassifier()
rfc_model.fit(X_train,y_train)

learning rate is greater than 1. You probably need to decrease learning rate.
learning rate is greater than 1. You probably need to decrease learning rate.


0:	learn: 0.8777079	total: 656us	remaining: 5.91ms
1:	learn: 10.1645479	total: 1.23ms	remaining: 4.92ms
2:	learn: 18.7255617	total: 1.79ms	remaining: 4.18ms
3:	learn: 13.1175414	total: 2.34ms	remaining: 3.51ms
4:	learn: 12.2357979	total: 2.9ms	remaining: 2.9ms
5:	learn: 11.3707882	total: 3.47ms	remaining: 2.31ms
6:	learn: 6.5238732	total: 4.01ms	remaining: 1.72ms
7:	learn: 4.8542079	total: 4.55ms	remaining: 1.14ms
8:	learn: 4.3347635	total: 5.08ms	remaining: 564us
9:	learn: 21.2496081	total: 5.62ms	remaining: 0us


RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

# Save Model to drive

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [173]:
# Save XGboost model to drive
XGB_model.save_model("/content/gdrive/MyDrive/ML_Model/Titanic_Proj_Model/XGBmodel.json",)
# Save CatBoost model to drive
CBC_model.save_model("/content/gdrive/MyDrive/ML_Model/Titanic_Proj_Model/Catmodel",format='cbm')
# Save RandomForest model to drive
joblib.dump(rfc_model, "/content/gdrive/MyDrive/ML_Model/Titanic_Proj_Model/random_forest.joblib")

['/content/gdrive/MyDrive/ML_Model/Titanic_Proj_Model/random_forest.joblib']